# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118945e+02     1.527839e+00
 * time: 0.08986711502075195
     1     1.019015e+01     9.885724e-01
 * time: 0.9487309455871582
     2    -1.125075e+01     1.077677e+00
 * time: 1.1982529163360596
     3    -3.410021e+01     8.017985e-01
 * time: 1.7034060955047607
     4    -4.733008e+01     6.307744e-01
 * time: 1.8351099491119385
     5    -5.683960e+01     1.852509e-01
 * time: 2.201050043106079
     6    -5.976698e+01     1.419984e-01
 * time: 2.313844919204712
     7    -6.091365e+01     5.171557e-02
 * time: 2.424452066421509
     8    -6.137835e+01     6.433255e-02
 * time: 2.5334579944610596
     9    -6.165878e+01     5.061342e-02
 * time: 2.6368050575256348
    10    -6.188756e+01     3.175499e-02
 * time: 2.748013973236084
    11    -6.203987e+01     2.055711e-02
 * time: 2.8561859130859375
    12    -6.210241e+01     1.539432e-02
 * time: 2.967139959335327
    13    -6.215219e+01     1.149073e-02
 * time: 3.07112812995910

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522630
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666462453
